In [1]:
import sympy as sym
import numpy as np
from sympy.tensor.tensor import TensorIndexType, tensor_indices, tensor_heads

In [2]:
from sympy.tensor.tensor import TensorIndexType, tensor_indices, TensorHead, riemann_cyclic, TensorSymmetry
Lorentz = TensorIndexType('Lorentz', dummy_name='L')
i, j, k, l = tensor_indices('i,j,k,l', Lorentz)
R = TensorHead('R', [Lorentz]*4, TensorSymmetry.riemann())
t = R(i,j,k,l)*(R(-i,-j,-k,-l) - 2*R(-i,-k,-j,-l))
riemann_cyclic(t)

0

In [18]:
a111,a112,a121,a122 = sym.symbols(r'a_{111} a_{112} a_{121} a_{122}')
a211,a212,a221,a222 = sym.symbols(r'a_{211} a_{212} a_{221} a_{222}')
b111,b112,b121,b122 = sym.symbols(r'b_{111} b_{112} b_{121} b_{122}')
b211,b212,b221,b222 = sym.symbols(r'b_{211} b_{212} b_{221} b_{222}')

d=2

A = np.array([
    [[a111,a112],
    [a121,a122]],
    [[a211,a212],
    [a221,a222]]
])
B = np.array([
    [[b111,b112],
    [b121,b122]],
    [[b211,b212],
    [b221,b222]]
])

def contraction21(A, B, d=2):
    out = np.zeros([d,d,d], dtype=object)
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for c1 in range(d):
                    for c2 in range(d):
                        out[i,j,k] += A[i][c1][c2] * B[c1, j ,c2, k]
                out[i,j,k] = out[i,j,k].simplify()
    return out

def contraction31(A, B, d=2):
    out = np.zeros([d,d,d], dtype=object)
    for i in range(d):
        for j in range(d):
            for k in range(d):
                out[i,j] = (A[i][k]+B[k][j]).simplify()

    return out

def TensorsMatrix22(A,B,M):
    out = np.zeros([d,d,d,d], dtype=object)
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    for c1 in range(d):
                        for c2 in range(d):
                            out[i,j,k,l]+= A[i,c1,j]*B[k, c2, l]*M[c1,c2]
                    out[i,j,k,l] = out[i,j,k,l].simplify()
    return out

def TensorsMatrix32(A,B,M):
    out = np.zeros([d,d,d,d], dtype=object)
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    for c1 in range(d):
                        for c2 in range(d):
                            out[i,j,k,l]+= A[i,j,c1]*B[k, c2, l]*M[c1,c2]
                    out[i,j,k,l] = out[i,j,k,l].simplify()
    return out

In [16]:
mu11, mu12, mu21, mu22 = sym.symbols(r'\mu_{11} \mu_{12} \mu_{21} \mu_{22}')
D1, D2 = sym.symbols(r'D_1 D_2')

T = np.array([
    [[mu11, mu12],
     [0,0]],
    [[0, 0],
     [mu21,mu22]]
])
D = np.array([
    [D1, 0],
    [0, D2]
])

In [17]:
newT = TensorsMatrix32(T,T,D)

In [19]:
final = contraction21(T, newT)

In [22]:
sym.Matrix(final[1])

Matrix([
[                                                 0,                                          0],
[\mu_{21}*(D_1*\mu_{11}*\mu_{21} + D_2*\mu_{22}**2), D_1*\mu_{12}*\mu_{21}**2 + D_2*\mu_{22}**3]])

In [30]:
def fullTensor(T,M):
    out = np.zeros([d,d,d,d,d,d,d,d,d,d,d], dtype=object)
    for i1 in range(d):
        for i2 in range(d):
            for i3 in range(d):
                for i4 in range(d):
                    for i5 in range(d):
                        for i6 in range(d):
                            for i7 in range(d):
                                for i8 in range(d):
                                    for i9 in range(d):
                                        for i10 in range(d):
                                            for i11 in range(d):
                                                out[i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11] = T[i1,i2,i3] * T[i4,i5,i6] * T[i7,i8,i9] * D[i10,i11]
    return out


In [33]:

A = fullTensor(T,D)
B = np.zeros([d,d,d,d,d,d,d,d,d], dtype=object)
for i1 in range(d):
    for i2 in range(d):
        for i3 in range(d):
            for i4 in range(d):
                for i5 in range(d):
                    for i6 in range(d):
                        for i7 in range(d):
                            for i8 in range(d):
                                for i9 in range(d):
                                    for c in range(d):
                                        B[i1,i2,i3,i4,i5,i6,i7,i8,i9] += A[i1,c,i2,c,i3,i4,i5,i6,i7,i8,i9]

A = np.zeros([d,d,d,d,d,d,d], dtype=object)
for i1 in range(d):
    for i2 in range(d):
        for i3 in range(d):
            for i4 in range(d):
                for i5 in range(d):
                    for i6 in range(d):
                        for i7 in range(d):
                            for c in range(d):
                                A[i1,i2,i3,i4,i5,i6,i7] += B[i1,c,i2,i3,c,i4,i5,i6,i7]

B = np.zeros([d,d,d,d,d], dtype=object)
for i1 in range(d):
    for i2 in range(d):
        for i3 in range(d):
            for i4 in range(d):
                for i5 in range(d):
                    for c in range(d):
                        B[i1,i2,i3,i4,i5] += A[i1,i2,c,i3,i4,c,i5]


out = np.zeros([d,d,d], dtype=object)
for i1 in range(d):
    for i2 in range(d):
        for i3 in range(d):
            for c in range(d):
                out[i1,i2,i3] += B[i1,i2,c,i3,c]
            out[i1,i2,i3].simplify()

sym.Matrix(out[0])

Matrix([
[D_1*\mu_{11}**3 + D_2*\mu_{12}**2*\mu_{21}, D_1*\mu_{11}**2*\mu_{12} + D_2*\mu_{12}**2*\mu_{22}],
[                                         0,                                                   0]])

In [88]:
def codification(T):
    out = np.zeros(d**11, dtype=object)
    for i1 in range(d):
        for i2 in range(d):
            for i3 in range(d):
                for i4 in range(d):
                    for i5 in range(d):
                        for i6 in range(d):
                            for i7 in range(d):
                                for i8 in range(d):
                                    for i9 in range(d):
                                        for i10 in range(d):
                                            for i11 in range(d):
                                                idx = i1 + i2*d + i3*d**2 + i4*d**3 + i5*d**4 + i6*d**5 + i7*d**6 + i8*d**7 + i9*d**8 + i10*d**9 + i11*d**10
                                                out[idx] = T[i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11]
    return out

def contraction(listT, index1, index2, dimout):
    out = np.zeros(d**dimout, dtype=object)

    for i in range(d**dimout):
        idx =  i % (d**(index1-1)) + i//(d**(index1-1)) * d**(index1)
        idx =  idx % (d**(index2-1)) + idx//(d**(index2-1)) * d**(index2)
        for c in range(d):
            out[i] += listT[idx + c* d**(index1-1) + c* d**(index2-1)]

    return out

def decodification(listT):
    out = np.zeros([d,d,d], dtype=object)
    for i in range(d):
        for j in range(d):
            for k in range(d):
                out[i,j,k] = listT[i+ j*d + k*d**2]
    return out
        

In [89]:
cod = codification(fullTensor(T,D))
cod = contraction(cod, 2, 4, 9)
cod = contraction(cod, 2, 5, 7)
cod = contraction(cod, 3, 6, 5)
cod = contraction(cod, 3, 5, 3)
final = decodification(cod)

In [93]:
sym.Matrix(final[1]) - sym.Matrix(out[1])

Matrix([
[0, 0],
[0, 0]])

In [109]:
def fullContraction(contractions):
    cod = codification(fullTensor(T,D))
    for i in range(4):
        i1,i2 = contractions[i]
        cod = contraction(cod, i1,i2, 9-2*i)
        print("se ejecutó: contraction(cod,{},{},{})".format(i1,i2, 9-2*i))
        for j in range(i,4):
            for k in range(2):
                if contractions[j][k] > i1 and contractions[j][k] > i2:
                    contractions[j][k]-=2
                elif contractions[j][k] > i1 or contractions[j][k] > i2:
                    contractions[j][k]-=1
        # print("pasó la iteración ", i)
        # print(contractions)
    return decodification(cod)


In [111]:
final2 = fullContraction([
    [2,4],[2,7],[6,10],[8,11]
])

se ejecutó: contraction(cod,2,4,9)
se ejecutó: contraction(cod,2,5,7)
se ejecutó: contraction(cod,3,6,5)
se ejecutó: contraction(cod,3,5,3)


In [112]:
sym.Matrix(final2[0]) - sym.Matrix(final[0])

Matrix([
[0, 0],
[0, 0]])

In [113]:
sym.Matrix(final2[0])

Matrix([
[D_1*\mu_{11}**3 + D_2*\mu_{12}**2*\mu_{21}, D_1*\mu_{11}**2*\mu_{12} + D_2*\mu_{12}**2*\mu_{22}],
[                                         0,                                                   0]])

In [68]:
i1,i2,i3,i4,i5 = 0,1,0,1,1

i = i1+i2*2+i3*2**2 + i4*2**3 + i5*2**4

index1=2

idx = i % (d**(index1-1)) + i//(d**(index1-1)) * d**(index1)
idx

52

In [70]:
i%(d**(index1-1))

0